In [1]:
from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 

os.getcwd()

os.listdir('/content/drive/MyDrive/Deeplabv3Flat')
os.listdir('/content/drive/MyDrive/Datasets')

os.chdir('/content/drive/MyDrive/Deeplabv3Flat')
print('Directory Changed')

Directory Changed


In [4]:
# camera-ready

import sys

! pip install import-ipynb
import import_ipynb

#sys.path.append("/content/drive/MyDrive/Deeplabv3Flat")
from datasets import DatasetVal # (this needs to be imported before torch, because cv2 needs to be imported before torch for some reason)

# Dont need becuase of the flat file structure
#sys.path.append("/content/drive/MyDrive/Deeplabv3Flat")
from deeplabv3 import DeepLabV3

#sys.path.append("/root/deeplabv3/utils")
from utils import label_img_to_color

import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import cv2

trainId_to_id = {
    0: 7,
    1: 8,
    2: 11,
    3: 12,
    4: 13,
    5: 17,
    6: 19,
    7: 20,
    8: 21,
    9: 22,
    10: 23,
    11: 24,
    12: 25,
    13: 26,
    14: 27,
    15: 28,
    16: 31,
    17: 32,
    18: 33,
    19: 0
}
trainId_to_id_map_func = np.vectorize(trainId_to_id.get)

batch_size = 2

best_trained_model = 'model_2_epoch_85.pth' # NB put the file name of best weight checkpoint 

network = DeepLabV3("eval_val_for_metrics", project_dir="/content/drive/MyDrive/Deeplabv3Flat").cuda()
network.load_state_dict(torch.load("/content/drive/MyDrive/Deeplabv3Flat/training_logs/model_2/checkpoints/" + best_trained_model))

val_dataset = DatasetVal(cityscapes_data_path="/content/drive/MyDrive/Datasets/Cityscapes",
                         cityscapes_meta_path="/content/drive/MyDrive/Datasets/Cityscapes/meta")

num_val_batches = int(len(val_dataset)/batch_size)
print ("num_val_batches:", num_val_batches)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=batch_size, shuffle=False,
                                         num_workers=1)

with open("/content/drive/MyDrive/Datasets/Cityscapes/meta/class_weights.pkl", "rb") as file: # (needed for python3)
    class_weights = np.array(pickle.load(file))
class_weights = torch.from_numpy(class_weights)
class_weights = Variable(class_weights.type(torch.FloatTensor)).cuda()

# loss function
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

network.eval() # (set in evaluation mode, this affects BatchNorm and dropout)
batch_losses = []
for step, (imgs, label_imgs, img_ids) in enumerate(val_loader):
    with torch.no_grad(): # (corresponds to setting volatile=True in all variables, this is done during inference to reduce memory consumption)
        imgs = Variable(imgs).cuda() # (shape: (batch_size, 3, img_h, img_w))
        label_imgs = Variable(label_imgs.type(torch.LongTensor)).cuda() # (shape: (batch_size, img_h, img_w))

        outputs = network(imgs) # (shape: (batch_size, num_classes, img_h, img_w))

        # compute the loss:
        loss = loss_fn(outputs, label_imgs)
        loss_value = loss.data.cpu().numpy()
        batch_losses.append(loss_value)

        ########################################################################
        # save data for visualization:
        ########################################################################
        outputs = F.upsample(outputs, size=(1024, 2048), mode="bilinear") # (shape: (batch_size, num_classes, 1024, 2048))

        outputs = outputs.data.cpu().numpy() # (shape: (batch_size, num_classes, 1024, 2048))
        pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, 1024, 2048))
        pred_label_imgs = pred_label_imgs.astype(np.uint8)

        for i in range(pred_label_imgs.shape[0]):
            pred_label_img = pred_label_imgs[i] # (shape: (1024, 2048))
            img_id = img_ids[i]

            # convert pred_label_img from trainId to id pixel values:
            pred_label_img = trainId_to_id_map_func(pred_label_img) # (shape: (1024, 2048))
            pred_label_img = pred_label_img.astype(np.uint8)

            cv2.imwrite(network.model_dir + "/" + img_id + "_pred_label_img.png", pred_label_img)

val_loss = np.mean(batch_losses)
print ("val loss: %g" % val_loss)


  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=20b5c4e6359604c5c0ea76ca74d4996a85cbed7269029b9f99e71435ea40c202
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
importing Jupyter notebook from datasets.ipynb
importing Jupyter notebook from deeplabv3.ipynb
importing Jupyter notebook from resnet.ipynb
importing Jupyter notebook from aspp.ipynb
importing Jupyter notebook from utils.ipynb
pretrained resnet, 18
num_val_batches: 250


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


val loss: 0.538642
